### Dataset Description
Breast cancer is one of the most common causes of death among women worldwide. Early detection helps in reducing the number of early deaths. The data reviews the medical images of breast cancer using ultrasound scan. Breast Ultrasound Dataset is categorized into three classes: normal, benign, and malignant images. Breast ultrasound images can produce great results in classification, detection, and segmentation of breast cancer when combined with machine learning.

The data collected at baseline include breast ultrasound images among women in ages between 25 and 75 years old. This data was collected in 2018. The number of patients is 600 female patients. The dataset consists of 780 images with an average image size of 500*500 pixels. The images are in PNG format. The ground truth images are presented with original images. The images are categorized into three classes, which are normal, benign, and malignant.

### Project overview
Speckle noise is a multiplicative noise that degrades the visual quality of ultrasound images and affects the clinical assessment. Denoising speckle is a very fundamental challenge in the field of Ultrasound imaging as it affects the clinical assessment. In this project, convolutional autoencoders are used to denoise speckle from breast ultrasound images. The results show that the under proper hyperparameter optimization implemented autoencoder performs significantly better than other traditional filters used for denoising.


# **1. Importing Libraries**

In [1]:
import re
import numpy as np 
import pandas as pd
import random
import math
import cv2 
import seaborn as sns
from glob import glob
import os
import itertools
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D,MaxPool2D ,UpSampling2D, Flatten, Input,LeakyReLU,BatchNormalization,Dropout
from tensorflow.nn import atrous_conv2d
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint, ReduceLROnPlateau, CSVLogger
import matplotlib.pyplot as plt
from skimage.util import random_noise
from skimage.metrics import peak_signal_noise_ratio as psnr, structural_similarity as ssim,mean_squared_error as mse
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# **2. Importing and preprocessing dataset**

In [3]:
path = '../input/breast-ultrasound-images-dataset/Dataset_BUSI_with_GT/*/*.png' 
images = glob(path)
images.sort()

**Comment** <br> 
Number of images for image and its masks are different for benign and malignant images. Some images have two masks.

In [4]:
#creating dictionary of images and masks
image = []
mask = []
count = 1
i= 0
num = len(images)
images_per_class = {}
while i < num-1:
    print(str(count) + "/" + str(num), end="\r")
    img = images[i]
    label = img.split('/')[-2] #get label from image name
    image_name = img.split('/')[-1].split(')')[0] #get image name
    if label not in images_per_class.keys():
        images_per_class[label] = {'image':[],'mask':[]} #make nested dictionory for each label
    
    
    if img.split('/')[-1][-5] == ')': #get the image
        image = cv2.resize(cv2.imread(img,cv2.IMREAD_GRAYSCALE),(128,128)) #read image
        images_per_class[label]['image'].append(image)
        mylist = images[i:]
        r = re.compile(r".*"+re.escape(image_name)+r"\)_mask*.") #look for mask
        masklist = list(filter(r.match, mylist))
        if len(masklist) == 1:
            mask = cv2.resize(cv2.imread(masklist[0]),(128,128))            
            images_per_class[label]['mask'].append(mask)
        else:  #there are two masks
            mask_1 = cv2.imread(masklist[0])
            mask_2 = cv2.imread(masklist[1])
            mask = cv2.resize(cv2.bitwise_or(mask_1,mask_2),(128,128))            
            images_per_class[label]['mask'].append(mask)      
        
    i+=1 
    
    count += 1

In [35]:
#Number of images per class
labels = []
for key,value in images_per_class.items():
    for k,v in value.items():
        labels.extend([k]*len(v))
        print("{0} : {1} : {2} ".format(key,k, len(v)))

In [36]:
fig,ax = plt.subplots(2,2)
ax[0,0].imshow(images_per_class['benign']['image'][50])
ax[0,0].set_title('Image')
ax[0,1].imshow(images_per_class['benign']['mask'][50])
ax[0,1].set_title('Mask')
ax[1,0].imshow(images_per_class['malignant']['image'][40])
ax[1,0].set_title('Image')
ax[1,1].imshow(images_per_class['malignant']['mask'][40])
ax[1,1].set_title('Mask')

# **3. Denoising Ultrasound Images**

### Creating dataset

In [7]:
#dataset for denoising consists of only images
dataset = images_per_class['benign']['image'] + images_per_class['malignant']['image'] + images_per_class['normal']['image']

In [8]:
#normalizing pixel values in the dataset
dataset = [img/255 for img in dataset] #for faster computation
random.shuffle(dataset)

In [9]:
#Plotting set of images
def plot_images(dataset):
  fig,ax=plt.subplots(1,5)
  fig.set_size_inches(40,20)
  for i in range(5,10):
    ax[i-5].imshow(dataset[i], cmap='gray')
  plt.show()

In [91]:
noised_dataset=[]
for img in dataset:
  noisy=random_noise(img,mode = 'speckle',var=0.04,mean=0) #introducing speckle noise
  noised_dataset.append(noisy)

noised_dataset=np.array(noised_dataset)

In [92]:
plot_images(dataset)

In [93]:
plot_images(noised_dataset)

In [94]:
#Splitting dataset for training and testing
x_train = noised_dataset[:546]
x_train_clean = dataset[:546]
x_val=noised_dataset[546:663]
x_val_clean=dataset[546:663]
x_test=noised_dataset[663:]
x_test_clean = dataset[663:]

In [95]:
x_train.shape

### Building model architecture

In [96]:
def denoising_autoencoder():
    i=Input(shape=(128,128,1))
    #encoder 
    x = Conv2D(128, (3,3), activation='relu', padding='same')(i)
    x = MaxPooling2D((2,2), padding='same')(x)
    x = Conv2D(128, (3,3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2,2), padding='same')(x)
    x = Conv2D(128, (3,3), activation='relu', padding='same')(x)
    x = Conv2D(128, (3,3), activation='relu', padding='same')(x)

    #decoder
    x = Conv2D(128, (3,3), activation='relu', padding='same')(x)
    x = Conv2D(128, (3,3), activation='relu', padding='same')(x)
    x = UpSampling2D((2,2))(x)
    x = Conv2D(128, (3,3), activation='relu', padding='same')(x)
    x = UpSampling2D((2,2))(x)
    x = Conv2D(1, (3,3), activation='sigmoid', padding='same')(x)

    #model
    autoencoder = Model(inputs=i, outputs=x)
    autoencoder.compile(optimizer='Adam', loss='mse')
    autoencoder.summary()

    return autoencoder


In [97]:
autoencoder = denoising_autoencoder()

In [25]:
# from keras.preprocessing.image import ImageDataGenerator

# datagen = ImageDataGenerator(
#     featurewise_center=True,
#     featurewise_std_normalization=True,
#     rotation_range=10,
#     fill_mode='nearest',
#     validation_split = 0.2
#     )

# datagen.fit(x_train)

# train_generator = datagen.flow(x_train, y_train, batch_size=64, subset='training')

# validation_generator = datagen.flow(X_train, y_train, batch_size=64, subset='validation')

### Training the  model

In [98]:
epochnum = 300
batchnum = 64

learning_rate_reduction = ReduceLROnPlateau(monitor='val_loss', 
                                            patience=8, 
                                            verbose=1, 
                                            factor=0.8, 
                                            min_lr=1e-10)
r = autoencoder.fit(x_train, x_train,
                epochs=epochnum,
                batch_size=batchnum,
                shuffle=True,
                validation_data=(x_val, x_val),
                callbacks=[learning_rate_reduction],verbose=0)
#model evaluation
print("Train score:", autoencoder.evaluate(x_train,x_train))
print("Test score:", autoencoder.evaluate(x_test,x_test))

### Get prediction

In [99]:
pred= autoencoder.predict(x_test[:5])

In [100]:
image_dict = {0 :x_test_clean,1:x_test,2:pred}
title = ['Original images','Images corrupted with speckle noise','Denoised images using autoencoder']

fig, axs = plt.subplots(nrows=3, ncols=1, figsize = (25,15),constrained_layout=True)
fig.suptitle('Noisy vs Denoised Images',fontsize = 20)

# clear subplots
for ax in axs:
    ax.remove()

# add subfigure per subplot
gridspec = axs[0].get_subplotspec().get_gridspec()
subfigs = [fig.add_subfigure(gs) for gs in gridspec]

for row, subfig in enumerate(subfigs):
    subfig.suptitle(title[row],fontsize=15)

    # create 1x3 subplots per subfig
    axs = subfig.subplots(nrows=1, ncols=5)
    for col, ax in enumerate(axs):
        ax.imshow(image_dict[row][col], aspect='auto', cmap='gray')

In [101]:
median_blur = cv2.medianBlur(np.float32(x_test[0]), (5))
gaussian_blur=cv2.GaussianBlur(x_test[0],(5,5),0)
average_blur=cv2.blur(x_test[0],(5,5))
bilateral_filter=cv2.bilateralFilter(np.float32(x_test[0]),9,75,75)
f,ax=plt.subplots(1,5)
f.set_size_inches(40,20)
ax[0].imshow(pred[0], cmap='gray')
ax[0].set_title('Autoencoder Image',fontsize = 15)
ax[1].imshow(median_blur,cmap='gray')
ax[1].set_title('Median Filter',fontsize = 15)
ax[2].imshow(gaussian_blur,cmap='gray')
ax[2].set_title('Gaussian Filter')
ax[3].imshow(average_blur,cmap='gray')
ax[3].set_title('Average Filter',fontsize = 15)
ax[4].imshow(bilateral_filter,cmap='gray')
ax[4].set_title('Bilateral Filter',fontsize = 15)
plt.show()

In [102]:
def plotLearningCurve(history,epochnum,batchnum):
  epochRange = range(1,epochnum+1)
  plt.figure(figsize = (5,5))
  plt.plot(epochRange,history.history['loss'],'b',label = 'Training Loss')
  plt.plot(epochRange,history.history['val_loss'],'r',label = 'Validation Loss')
  plt.xlabel('Epoch', fontsize = 15)
  plt.ylabel('Loss', fontsize = 15)
  plt.grid(color='gray', linestyle='--')
  plt.legend()
  plt.title('LOSS, Epochs={}, Batch={}'.format(epochnum, batchnum))
  plt.show()

In [103]:
plotLearningCurve(r,epochnum,batchnum)

In [104]:
value1 = psnr(x_test[0], pred[0].reshape(x_test_clean[0].shape[0],-1))
value2 = psnr(x_test[0], median_blur)
value3 = psnr(x_test[0], gaussian_blur)
value4 = psnr(x_test[0], average_blur)
value5 = psnr(x_test[0], bilateral_filter)
metric = pd.DataFrame({'Autoencoder':value1,'Median Blur':value2,'Gaussian Blur':value3,'Average Blur':value4,'Bilateral Filter':value5},index = ['PSNR'])
value1 = mse(x_test[0], pred[0].reshape(x_test_clean[0].shape[0],-1))
value2 = mse(x_test[0], median_blur)
value3 = mse(x_test[0], gaussian_blur)
value4 = mse(x_test[0], average_blur)
value5 = mse(x_test[0], bilateral_filter)
metric.loc['MSE'] = [value1,value2,value3,value4,value5]
value1 = ssim(x_test[0], pred[0].reshape(x_test_clean[0].shape[0],-1))
value2 = ssim(x_test[0], median_blur)
value3 = ssim(x_test[0], gaussian_blur)
value4 = ssim(x_test[0], average_blur)
value5 = ssim(x_test[0], bilateral_filter)
metric.loc['SSIM'] = [value1,value2,value3,value4,value5]
metric


In [105]:
# Importing libraries
import matplotlib.pyplot as plt
import numpy as np
import math

# Using Numpy to create an array X
X = [0.04,0.08,0.12,0.16,0.20]

# Assign variables to the y axis part of the curve
a =[21.4778,19.3744,18.6074,16.8753,16.7291]
b =[19.2485,17.2483,15.8877,15.1128,14.3162]
c =[20.4853,18.4170,17.0090,16.2425,15.4575]
d =[19.1708,17.2985,15.9729,15.2356,14.4902]
e =[18.6800,16.9777,15.7149,14.9955,14.2977]

# Plotting both the curves simultaneously
plt.plot(X, a, color='r', label='Autoencoder')
plt.plot(X, b, color='g', label='Median Blur')
plt.plot(X, c, color='b', label='Gaussian Blur')
plt.plot(X, d, color='y', label='Average Blur')
plt.plot(X, e, color='c', label='Bilateral Filter')

# Naming the x-axis, y-axis and the whole graph
plt.xlabel("Variance")
plt.ylabel("PSNR")
plt.title("PSNR comparision")

# Adding legend, which helps us recognize the curve according to it's color
plt.legend()

# To load the display window
plt.show()


In [106]:
# Importing libraries
import matplotlib.pyplot as plt
import numpy as np
import math

# Using Numpy to create an array X
X = [0.04,0.08,0.12,0.16,0.20]

# Assign variables to the y axis part of the curve
a =[0.0071,0.0115,0.0137,0.0205,0.0212]
b =[0.0118,0.0188,0.0257,0.0308,0.0370]
c =[0.0089,0.0143,0.0199,0.0237,0.0284]
d =[0.0121,0.0186,0.0252,0.0299,0.0355]
e =[0.0135,0.0200,0.0268,0.0316,0.0371]

# Plotting both the curves simultaneously
plt.plot(X, a, color='r', label='Autoencoder')
plt.plot(X, b, color='g', label='Median Blur')
plt.plot(X, c, color='b', label='Gaussian Blur')
plt.plot(X, d, color='y', label='Average Blur')
plt.plot(X, e, color='c', label='Bilateral Filter')

# Naming the x-axis, y-axis and the whole graph
plt.xlabel("Variance")
plt.ylabel("MSE")
plt.title("MSE comparision")

# Adding legend, which helps us recognize the curve according to it's color
plt.legend()

# To load the display window
plt.show()


In [108]:
# Importing libraries
import matplotlib.pyplot as plt
import numpy as np
import math

# Using Numpy to create an array X
X = [0.04,0.08,0.12,0.16,0.20]

# Assign variables to the y axis part of the curve
a =[0.7531,0.7230,0.7242,0.6152,0.6656]
b =[0.5408,0.4415,0.3831,0.3507,0.3153]
c =[0.6575,0.5760,0.5228,0.4949,0.4651]
d =[0.5280,0.4344,0.3770,0.3427,0.3092]
e =[0.4735,0.3858,0.3314,0.2972,0.2671]

# Plotting both the curves simultaneously
plt.plot(X, a, color='r', label='Autoencoder')
plt.plot(X, b, color='g', label='Median Blur')
plt.plot(X, c, color='b', label='Gaussian Blur')
plt.plot(X, d, color='y', label='Average Blur')
plt.plot(X, e, color='c', label='Bilateral Filter')

# Naming the x-axis, y-axis and the whole graph
plt.xlabel("Variance")
plt.ylabel("SSIM")
plt.title("SSIM comparision")

# Adding legend, which helps us recognize the curve according to it's color
plt.legend()

# To load the display window
plt.show()
